In [295]:
import numpy as np
import pandas as pd

import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


from scipy import stats
from sklearn import preprocessing 
from sklearn.metrics.scorer import make_scorer
from sklearn.model_selection import GridSearchCV
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols

import matplotlib.pyplot as plt 
import seaborn as sns
from pandas import Timestamp

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.svm import SVR

import xgboost as xgb
from xgboost import XGBRegressor 
from xgboost.sklearn import XGBClassifier # sklearn’s Grid Search with parallel processing
from xgboost import plot_importance
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from catboost import CatBoostRegressor

# import warnings
# warnings.filterwarnings('ignore')

In [296]:
data = pd.read_csv('DF_File_sample.csv')
data['SKU_Customer'] = data['DemandCustomer'] + data['SKU10']
data.drop(['DemandCustomer', 'SKU10'], axis =1, inplace = True)

### LabelEncoding:

In [166]:
data.columns

Index(['Brand', 'Year', 'Quarter', 'Month_No', 'Week_No', 'Sales', 'W_Nielsen',
       'SKU_Customer'],
      dtype='object')

In [297]:
cat_var = [data.columns.get_loc(c) for c in data.columns if data.loc[:, c].dtypes=='object']

In [175]:
data_LE.iloc[:, 0]

0        ARW
1        ARW
2        ARW
3        ARW
4        ARW
        ... 
15067    LYS
15068    LYS
15069    LYS
15070    LYS
15071    LYS
Name: Brand, Length: 15072, dtype: object

In [298]:
data_LE = data.copy()
le = preprocessing.LabelEncoder()

for i in cat_var:
    data_LE.iloc[:, i] = le.fit_transform(data_LE.iloc[:, i])

In [299]:
data = data_LE

### One Hot Encoder:

In [226]:
data_OH = pd.get_dummies(data)

In [227]:
data = data_OH.copy()

### Data cleaning:

In [300]:
data['year_week'] = data['Year'].astype(str) + '-' + data['Week_No'].astype(str)
data['pre_date'] = data['year_week'].apply(lambda x: datetime.datetime.strptime(x + '-4',  "%G-%V-%w"))

In [301]:
first_null = data.groupby('pre_date').Sales.sum().loc[lambda x: x == 0].sort_values().index[0]
data = data[data.pre_date < first_null]

In [302]:
train = data[data['pre_date'] < pd.Series(sorted(data['pre_date'].unique())).iloc[-26]]
data.drop(['pre_date', 'year_week'], axis =1, inplace = True)
train.drop(['pre_date', 'year_week'], axis =1, inplace = True)
test = data.loc[set(data.index) - set(train.index), :]

In [303]:
X_train, X_test, y_train, y_test = train_test_split(train.loc[:, train.columns!='Sales'], train['Sales'], test_size=0.33, random_state=42)

In [230]:
X_train_OH, X_test_OH, y_train_OH, y_test_OH = train_test_split(train.loc[:, train.columns!='Sales'], train['Sales'], test_size=0.33, random_state=42)

### Data normalization:

In [304]:
norm_scaler = preprocessing.StandardScaler()
min_max_scaler = preprocessing.MinMaxScaler() # [0, 1]
max_abs_scaler = preprocessing.MaxAbsScaler() # [-1, 1]

X_train_scaled = norm_scaler.fit_transform(X_train)
X_test_scaled = norm_scaler.transform(X_test)

X_train_scaled_OH = norm_scaler.fit_transform(X_train_OH)
X_test_scaled_OH = norm_scaler.transform(X_test_OH)

# Modeling:

In [305]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return (np.mean(np.abs((y_true - y_pred))) * 100 / np.mean(np.abs((y_true)))) 

## Ridge:

In [290]:
grid = {"alpha" : np.arange(0, 2, 0.001)}
model = Ridge() 
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=True)

ridge_cv = GridSearchCV(model, grid, scoring = scorer, n_jobs=1)
ridge_cv.fit(X_train_scaled, y_train, cat_features= cat_var)

print("ridge mape:", mean_absolute_percentage_error(y_test, ridge_cv.predict(X_test_scaled)), ridge_cv.best_params_)

TypeError: fit() got an unexpected keyword argument 'cat_features'

## LightGBM:

In [306]:
param_grid = {
    'n_estimators' : list(range(150, 400, 50)),
    'num_leaves': list(range(8, 16, 4)),
    # 'min_data_in_leaf': [10, 20, 40, 60, 100],
    'max_depth': [8, 12, 15],
    'learning_rate': [.3, 0.1, 0.01]}
    # 'bagging_freq': [3, 4, 5, 6, 7],
    # 'bagging_fraction': np.linspace(0.6, 0.95, 10),
    # 'reg_alpha': np.linspace(0.1, 0.95, 10),
    # 'reg_lambda': np.linspace(0.1, 0.95, 10)
                                                  
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better = False)   
gbm = GridSearchCV(lgb.LGBMRegressor(categorical_feature = cat_var), 
                   param_grid, 
                   scoring = scorer, n_jobs=1)


gbm.fit(X_train_scaled, y_train)


print("lightgbm mape:", mean_absolute_percentage_error(y_test, gbm.predict(X_test_scaled)), gbm.best_params_)

lightgbm mape: 48.13006755298951 {'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 200, 'num_leaves': 12}


In [255]:
param_grid = {
    'n_estimators' : list(range(150, 350, 50)),
    'num_leaves': list(range(10,14,1)),
    # 'min_data_in_leaf': [10, 20, 40, 60, 100],
    'max_depth': [8, 12, 15],
    'learning_rate': [.3, 0.1, 0.05]}
    # 'bagging_freq': [3, 4, 5, 6, 7],
    # 'bagging_fraction': np.linspace(0.6, 0.95, 10),
    # 'reg_alpha': np.linspace(0.1, 0.95, 10),
    # 'reg_lambda': np.linspace(0.1, 0.95, 10)
                          
model = lgb.LGBMRegressor(categorical_feature = cat_var),                         
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better = False) 
    
gbm = GridSearchCV(lgb.LGBMRegressor(categorical_feature = cat_var), 
                   param_grid, 
                   scoring = scorer, 
                   n_jobs=1,
                   verbose=2)


gbm.fit(X_train_scaled, y_train)


print("lightgbm mape:", mean_absolute_percentage_error(y_test, gbm.predict(X_test_scaled)), gbm.best_params_)

Fitting 3 folds for each of 144 candidates, totalling 432 fits
[CV] learning_rate=0.3, max_depth=8, n_estimators=150, num_leaves=10 .
[CV]  learning_rate=0.3, max_depth=8, n_estimators=150, num_leaves=10, total=   0.1s
[CV] learning_rate=0.3, max_depth=8, n_estimators=150, num_leaves=10 .
[CV]  learning_rate=0.3, max_depth=8, n_estimators=150, num_leaves=10, total=   0.1s
[CV] learning_rate=0.3, max_depth=8, n_estimators=150, num_leaves=10 .


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  learning_rate=0.3, max_depth=8, n_estimators=150, num_leaves=10, total=   0.1s
[CV] learning_rate=0.3, max_depth=8, n_estimators=150, num_leaves=11 .
[CV]  learning_rate=0.3, max_depth=8, n_estimators=150, num_leaves=11, total=   0.1s
[CV] learning_rate=0.3, max_depth=8, n_estimators=150, num_leaves=11 .
[CV]  learning_rate=0.3, max_depth=8, n_estimators=150, num_leaves=11, total=   0.1s
[CV] learning_rate=0.3, max_depth=8, n_estimators=150, num_leaves=11 .
[CV]  learning_rate=0.3, max_depth=8, n_estimators=150, num_leaves=11, total=   0.1s
[CV] learning_rate=0.3, max_depth=8, n_estimators=150, num_leaves=12 .
[CV]  learning_rate=0.3, max_depth=8, n_estimators=150, num_leaves=12, total=   0.1s
[CV] learning_rate=0.3, max_depth=8, n_estimators=150, num_leaves=12 .
[CV]  learning_rate=0.3, max_depth=8, n_estimators=150, num_leaves=12, total=   0.1s
[CV] learning_rate=0.3, max_depth=8, n_estimators=150, num_leaves=12 .
[CV]  learning_rate=0.3, max_depth=8, n_estimators=150, num_leav

[CV]  learning_rate=0.3, max_depth=12, n_estimators=150, num_leaves=12, total=   0.1s
[CV] learning_rate=0.3, max_depth=12, n_estimators=150, num_leaves=13 
[CV]  learning_rate=0.3, max_depth=12, n_estimators=150, num_leaves=13, total=   0.1s
[CV] learning_rate=0.3, max_depth=12, n_estimators=150, num_leaves=13 
[CV]  learning_rate=0.3, max_depth=12, n_estimators=150, num_leaves=13, total=   0.1s
[CV] learning_rate=0.3, max_depth=12, n_estimators=150, num_leaves=13 
[CV]  learning_rate=0.3, max_depth=12, n_estimators=150, num_leaves=13, total=   0.1s
[CV] learning_rate=0.3, max_depth=12, n_estimators=200, num_leaves=10 
[CV]  learning_rate=0.3, max_depth=12, n_estimators=200, num_leaves=10, total=   0.1s
[CV] learning_rate=0.3, max_depth=12, n_estimators=200, num_leaves=10 
[CV]  learning_rate=0.3, max_depth=12, n_estimators=200, num_leaves=10, total=   0.1s
[CV] learning_rate=0.3, max_depth=12, n_estimators=200, num_leaves=10 
[CV]  learning_rate=0.3, max_depth=12, n_estimators=200, n

[CV]  learning_rate=0.3, max_depth=15, n_estimators=200, num_leaves=11, total=   0.1s
[CV] learning_rate=0.3, max_depth=15, n_estimators=200, num_leaves=11 
[CV]  learning_rate=0.3, max_depth=15, n_estimators=200, num_leaves=11, total=   0.1s
[CV] learning_rate=0.3, max_depth=15, n_estimators=200, num_leaves=11 
[CV]  learning_rate=0.3, max_depth=15, n_estimators=200, num_leaves=11, total=   0.1s
[CV] learning_rate=0.3, max_depth=15, n_estimators=200, num_leaves=12 
[CV]  learning_rate=0.3, max_depth=15, n_estimators=200, num_leaves=12, total=   0.1s
[CV] learning_rate=0.3, max_depth=15, n_estimators=200, num_leaves=12 
[CV]  learning_rate=0.3, max_depth=15, n_estimators=200, num_leaves=12, total=   0.1s
[CV] learning_rate=0.3, max_depth=15, n_estimators=200, num_leaves=12 
[CV]  learning_rate=0.3, max_depth=15, n_estimators=200, num_leaves=12, total=   0.1s
[CV] learning_rate=0.3, max_depth=15, n_estimators=200, num_leaves=13 
[CV]  learning_rate=0.3, max_depth=15, n_estimators=200, n

...

[CV]  learning_rate=0.1, max_depth=15, n_estimators=250, num_leaves=12, total=   0.1s
[CV] learning_rate=0.1, max_depth=15, n_estimators=250, num_leaves=12 
[CV]  learning_rate=0.1, max_depth=15, n_estimators=250, num_leaves=12, total=   0.1s
[CV] learning_rate=0.1, max_depth=15, n_estimators=250, num_leaves=13 
[CV]  learning_rate=0.1, max_depth=15, n_estimators=250, num_leaves=13, total=   0.1s
[CV] learning_rate=0.1, max_depth=15, n_estimators=250, num_leaves=13 
[CV]  learning_rate=0.1, max_depth=15, n_estimators=250, num_leaves=13, total=   0.1s
[CV] learning_rate=0.1, max_depth=15, n_estimators=250, num_leaves=13 
[CV]  learning_rate=0.1, max_depth=15, n_estimators=250, num_leaves=13, total=   0.1s
[CV] learning_rate=0.1, max_depth=15, n_estimators=300, num_leaves=10 
[CV]  learning_rate=0.1, max_depth=15, n_estimators=300, num_leaves=10, total=   0.1s
[CV] learning_rate=0.1, max_depth=15, n_estimators=300, num_leaves=10 
[CV]  learning_rate=0.1, max_depth=15, n_estimators=300, n

[CV]  learning_rate=0.05, max_depth=8, n_estimators=300, num_leaves=10, total=   0.1s
[CV] learning_rate=0.05, max_depth=8, n_estimators=300, num_leaves=10 
[CV]  learning_rate=0.05, max_depth=8, n_estimators=300, num_leaves=10, total=   0.1s
[CV] learning_rate=0.05, max_depth=8, n_estimators=300, num_leaves=10 
[CV]  learning_rate=0.05, max_depth=8, n_estimators=300, num_leaves=10, total=   0.1s
[CV] learning_rate=0.05, max_depth=8, n_estimators=300, num_leaves=11 
[CV]  learning_rate=0.05, max_depth=8, n_estimators=300, num_leaves=11, total=   0.1s
[CV] learning_rate=0.05, max_depth=8, n_estimators=300, num_leaves=11 
[CV]  learning_rate=0.05, max_depth=8, n_estimators=300, num_leaves=11, total=   0.1s
[CV] learning_rate=0.05, max_depth=8, n_estimators=300, num_leaves=11 
[CV]  learning_rate=0.05, max_depth=8, n_estimators=300, num_leaves=11, total=   0.1s
[CV] learning_rate=0.05, max_depth=8, n_estimators=300, num_leaves=12 
[CV]  learning_rate=0.05, max_depth=8, n_estimators=300, n

[CV]  learning_rate=0.05, max_depth=12, n_estimators=300, num_leaves=11, total=   0.2s
[CV] learning_rate=0.05, max_depth=12, n_estimators=300, num_leaves=11 
[CV]  learning_rate=0.05, max_depth=12, n_estimators=300, num_leaves=11, total=   0.2s
[CV] learning_rate=0.05, max_depth=12, n_estimators=300, num_leaves=12 
[CV]  learning_rate=0.05, max_depth=12, n_estimators=300, num_leaves=12, total=   0.2s
[CV] learning_rate=0.05, max_depth=12, n_estimators=300, num_leaves=12 
[CV]  learning_rate=0.05, max_depth=12, n_estimators=300, num_leaves=12, total=   0.2s
[CV] learning_rate=0.05, max_depth=12, n_estimators=300, num_leaves=12 
[CV]  learning_rate=0.05, max_depth=12, n_estimators=300, num_leaves=12, total=   0.2s
[CV] learning_rate=0.05, max_depth=12, n_estimators=300, num_leaves=13 
[CV]  learning_rate=0.05, max_depth=12, n_estimators=300, num_leaves=13, total=   0.2s
[CV] learning_rate=0.05, max_depth=12, n_estimators=300, num_leaves=13 
[CV]  learning_rate=0.05, max_depth=12, n_esti

[CV]  learning_rate=0.05, max_depth=15, n_estimators=300, num_leaves=13, total=   0.2s
[CV] learning_rate=0.05, max_depth=15, n_estimators=300, num_leaves=13 
[CV]  learning_rate=0.05, max_depth=15, n_estimators=300, num_leaves=13, total=   0.2s
[CV] learning_rate=0.05, max_depth=15, n_estimators=300, num_leaves=13 
[CV]  learning_rate=0.05, max_depth=15, n_estimators=300, num_leaves=13, total=   0.2s
lightgbm mape: 48.524358801238066 {'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 200, 'num_leaves': 13}


[Parallel(n_jobs=1)]: Done 432 out of 432 | elapsed:   51.2s finished


## tuning other param:

In [256]:
param_grid = {
    'n_estimators' : [200],
    'num_leaves': [13],
    # 'min_data_in_leaf': [10, 30, 60],
    'max_depth': [8],
    'learning_rate':  [0.1],
    # 'bagging_freq': [3, 4, 5, 6, 7],
    # 'bagging_fraction': np.linspace(0.6, 0.95, 10),
    'reg_alpha': np.linspace(0.1, 0.95, 10),
    'reg_lambda': np.linspace(0.1, 0.95, 10)}
                          
model = lgb.LGBMRegressor(categorical_feature = cat_var),                         
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better = False) 
    
gbm = GridSearchCV(lgb.LGBMRegressor(categorical_feature = cat_var), 
                   param_grid, 
                   scoring = scorer, 
                   n_jobs=1,
                   verbose=2)


gbm.fit(X_train_scaled, y_train)


print("lightgbm mape:", mean_absolute_percentage_error(y_test, gbm.predict(X_test_scaled)), gbm.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.1 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.1, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.1, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.1 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.1, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.19444444444444445 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.19444444444444445, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.19444444444444445 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.19444444444444445, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.19444444444444445 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=2

[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.19444444444444445, reg_lambda=0.19444444444444445, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.19444444444444445, reg_lambda=0.28888888888888886 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.19444444444444445, reg_lambda=0.28888888888888886, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.19444444444444445, reg_lambda=0.28888888888888886 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.19444444444444445, reg_lambda=0.28888888888888886, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.19444444444444445, reg_lambda=0.28888888888888886 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.19444444444444445, reg_lambda=0.28888888888888886, total=   0.1s
[CV] lear

[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.28888888888888886, reg_lambda=0.19444444444444445, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.28888888888888886, reg_lambda=0.28888888888888886 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.28888888888888886, reg_lambda=0.28888888888888886, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.28888888888888886, reg_lambda=0.28888888888888886 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.28888888888888886, reg_lambda=0.28888888888888886, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.28888888888888886, reg_lambda=0.28888888888888886 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.28888888888888886, reg_lambda=0.28888888888888886, total=   0.1s
[CV] lear

...

[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.6666666666666666, reg_lambda=0.3833333333333333, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.6666666666666666, reg_lambda=0.3833333333333333 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.6666666666666666, reg_lambda=0.3833333333333333, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.6666666666666666, reg_lambda=0.3833333333333333 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.6666666666666666, reg_lambda=0.3833333333333333, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.6666666666666666, reg_lambda=0.47777777777777775 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.6666666666666666, reg_lambda=0.47777777777777775, total=   0.1s
[CV] learning_rate=0.

[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.7611111111111111, reg_lambda=0.3833333333333333, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.7611111111111111, reg_lambda=0.47777777777777775 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.7611111111111111, reg_lambda=0.47777777777777775, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.7611111111111111, reg_lambda=0.47777777777777775 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.7611111111111111, reg_lambda=0.47777777777777775, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.7611111111111111, reg_lambda=0.47777777777777775 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.7611111111111111, reg_lambda=0.47777777777777775, total=   0.1s
[CV] learning_rat

[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.8555555555555555, reg_lambda=0.47777777777777775, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.8555555555555555, reg_lambda=0.47777777777777775 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.8555555555555555, reg_lambda=0.47777777777777775, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.8555555555555555, reg_lambda=0.5722222222222222 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.8555555555555555, reg_lambda=0.5722222222222222, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.8555555555555555, reg_lambda=0.5722222222222222 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.8555555555555555, reg_lambda=0.5722222222222222, total=   0.1s
[CV] learning_rate=0

[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.95, reg_lambda=0.5722222222222222, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.95, reg_lambda=0.5722222222222222 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.95, reg_lambda=0.5722222222222222, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.95, reg_lambda=0.5722222222222222 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.95, reg_lambda=0.5722222222222222, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.95, reg_lambda=0.6666666666666666 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.95, reg_lambda=0.6666666666666666, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.95, reg_lambda=0.6666666666666666 
[CV]

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:   33.9s finished


### categorical_feature norm or not:

In [265]:

param = {'num_leaves':13, 'max_depth':8,'learning_rate':.1,'max_bin':200}
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better = False) 
    
gbm2 = GridSearchCV(lgb.LGBMRegressor(), 
                   param_grid, 
                   scoring = scorer, 
                   n_jobs=1,
                   verbose=2)

train_data = lgb.Dataset(X_train, label=y_train, categorical_feature = cat_var)
test_data = lgb.Dataset(X_test, label=y_test, categorical_feature = cat_var)

gbm2.fit(X_train, y_train)
gbm2 = lgb.train(param, train_data,valid_sets=test_data, num_boost_round=5000)


Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.1 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.1, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.1, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.1 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.1, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.19444444444444445 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.19444444444444445, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.19444444444444445 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.19444444444444445, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.1, reg_lambda=0.19444444444444445 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=2

[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.19444444444444445, reg_lambda=0.19444444444444445, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.19444444444444445, reg_lambda=0.28888888888888886 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.19444444444444445, reg_lambda=0.28888888888888886, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.19444444444444445, reg_lambda=0.28888888888888886 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.19444444444444445, reg_lambda=0.28888888888888886, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.19444444444444445, reg_lambda=0.28888888888888886 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.19444444444444445, reg_lambda=0.28888888888888886, total=   0.1s
[CV] lear

[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.28888888888888886, reg_lambda=0.19444444444444445, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.28888888888888886, reg_lambda=0.28888888888888886 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.28888888888888886, reg_lambda=0.28888888888888886, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.28888888888888886, reg_lambda=0.28888888888888886 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.28888888888888886, reg_lambda=0.28888888888888886, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.28888888888888886, reg_lambda=0.28888888888888886 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.28888888888888886, reg_lambda=0.28888888888888886, total=   0.1s
[CV] lear

...

[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.7611111111111111, reg_lambda=0.3833333333333333, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.7611111111111111, reg_lambda=0.47777777777777775 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.7611111111111111, reg_lambda=0.47777777777777775, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.7611111111111111, reg_lambda=0.47777777777777775 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.7611111111111111, reg_lambda=0.47777777777777775, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.7611111111111111, reg_lambda=0.47777777777777775 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.7611111111111111, reg_lambda=0.47777777777777775, total=   0.1s
[CV] learning_rat

[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.8555555555555555, reg_lambda=0.47777777777777775, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.8555555555555555, reg_lambda=0.47777777777777775 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.8555555555555555, reg_lambda=0.47777777777777775, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.8555555555555555, reg_lambda=0.5722222222222222 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.8555555555555555, reg_lambda=0.5722222222222222, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.8555555555555555, reg_lambda=0.5722222222222222 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.8555555555555555, reg_lambda=0.5722222222222222, total=   0.1s
[CV] learning_rate=0

[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.95, reg_lambda=0.5722222222222222, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.95, reg_lambda=0.5722222222222222 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.95, reg_lambda=0.5722222222222222, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.95, reg_lambda=0.5722222222222222 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.95, reg_lambda=0.5722222222222222, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.95, reg_lambda=0.6666666666666666 
[CV]  learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.95, reg_lambda=0.6666666666666666, total=   0.1s
[CV] learning_rate=0.1, max_depth=8, n_estimators=200, num_leaves=13, reg_alpha=0.95, reg_lambda=0.6666666666666666 
[CV]

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:   35.9s finished


ValueError: For early stopping, at least one dataset and eval metric is required for evaluation

In [264]:
print("lightgbm mape:", mean_absolute_percentage_error(y_test, gbm2.predict(X_test_scaled)), gbm2.best_params_)

lightgbm mape: 181.3898981448889 {'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 200, 'num_leaves': 13, 'reg_alpha': 0.1, 'reg_lambda': 0.6666666666666666}


# XGB:

In [244]:
param_grid = {
    'n_estimators' : list(range(150, 400, 50)),
    # 'num_leaves': list(range(8, 16, 4)),
    # 'min_data_in_leaf': [10, 20, 40, 60, 100],
    'max_depth': [8, 12, 15],
    'learning_rate': [.3, 0.1, 0.01]}
    # 'bagging_freq': [3, 4, 5, 6, 7],
    # 'bagging_fraction': np.linspace(0.6, 0.95, 10),
    # 'reg_alpha': np.linspace(0.1, 0.95, 10),
    # 'reg_lambda': np.linspace(0.1, 0.95, 10)
                          
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better = False) 
model_xgb = xgb.XGBRegressor(objective = 'reg:squarederror')                        

    
grid_xgb = GridSearchCV(model_xgb, 
                        param_grid, 
                        scoring = scorer,
                        n_jobs=1,
                        pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
                        verbose=2)

grid_xgb.fit(X_train_scaled_OH, y_train)

Fitting 3 folds for each of 45 candidates, totalling 135 fits
[CV] learning_rate=0.3, max_depth=8, n_estimators=150 ................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] . learning_rate=0.3, max_depth=8, n_estimators=150, total=   1.3s
[CV] learning_rate=0.3, max_depth=8, n_estimators=150 ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s


[CV] . learning_rate=0.3, max_depth=8, n_estimators=150, total=   1.3s
[CV] learning_rate=0.3, max_depth=8, n_estimators=150 ................
[CV] . learning_rate=0.3, max_depth=8, n_estimators=150, total=   1.2s
[CV] learning_rate=0.3, max_depth=8, n_estimators=200 ................
[CV] . learning_rate=0.3, max_depth=8, n_estimators=200, total=   1.7s
[CV] learning_rate=0.3, max_depth=8, n_estimators=200 ................
[CV] . learning_rate=0.3, max_depth=8, n_estimators=200, total=   1.7s
[CV] learning_rate=0.3, max_depth=8, n_estimators=200 ................
[CV] . learning_rate=0.3, max_depth=8, n_estimators=200, total=   1.8s
[CV] learning_rate=0.3, max_depth=8, n_estimators=250 ................
[CV] . learning_rate=0.3, max_depth=8, n_estimators=250, total=   2.1s
[CV] learning_rate=0.3, max_depth=8, n_estimators=250 ................
[CV] . learning_rate=0.3, max_depth=8, n_estimators=250, total=   2.2s
[CV] learning_rate=0.3, max_depth=8, n_estimators=250 ................
[CV] .

[CV] . learning_rate=0.1, max_depth=8, n_estimators=350, total=   3.0s
[CV] learning_rate=0.1, max_depth=12, n_estimators=150 ...............
[CV]  learning_rate=0.1, max_depth=12, n_estimators=150, total=   2.0s
[CV] learning_rate=0.1, max_depth=12, n_estimators=150 ...............
[CV]  learning_rate=0.1, max_depth=12, n_estimators=150, total=   2.0s
[CV] learning_rate=0.1, max_depth=12, n_estimators=150 ...............
[CV]  learning_rate=0.1, max_depth=12, n_estimators=150, total=   2.0s
[CV] learning_rate=0.1, max_depth=12, n_estimators=200 ...............
[CV]  learning_rate=0.1, max_depth=12, n_estimators=200, total=   2.6s
[CV] learning_rate=0.1, max_depth=12, n_estimators=200 ...............
[CV]  learning_rate=0.1, max_depth=12, n_estimators=200, total=   2.6s
[CV] learning_rate=0.1, max_depth=12, n_estimators=200 ...............
[CV]  learning_rate=0.1, max_depth=12, n_estimators=200, total=   2.7s
[CV] learning_rate=0.1, max_depth=12, n_estimators=250 ...............
[CV]  

[CV]  learning_rate=0.01, max_depth=12, n_estimators=350, total=   4.8s
[CV] learning_rate=0.01, max_depth=12, n_estimators=350 ..............
[CV]  learning_rate=0.01, max_depth=12, n_estimators=350, total=   4.9s
[CV] learning_rate=0.01, max_depth=12, n_estimators=350 ..............
[CV]  learning_rate=0.01, max_depth=12, n_estimators=350, total=   4.8s
[CV] learning_rate=0.01, max_depth=15, n_estimators=150 ..............
[CV]  learning_rate=0.01, max_depth=15, n_estimators=150, total=   2.7s
[CV] learning_rate=0.01, max_depth=15, n_estimators=150 ..............
[CV]  learning_rate=0.01, max_depth=15, n_estimators=150, total=   2.6s
[CV] learning_rate=0.01, max_depth=15, n_estimators=150 ..............
[CV]  learning_rate=0.01, max_depth=15, n_estimators=150, total=   2.7s
[CV] learning_rate=0.01, max_depth=15, n_estimators=200 ..............
[CV]  learning_rate=0.01, max_depth=15, n_estimators=200, total=   3.6s
[CV] learning_rate=0.01, max_depth=15, n_estimators=200 ..............

[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:  7.7min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:squarederror',
                                    rand...=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='warn', n_jobs=1,
             param_grid={'learning_rate': [0.3, 0.1, 0.01],
                         'max_depth': [8, 12, 15],
                     

In [246]:
print("lightgbm mape:", mean_absolute_percentage_error(y_test, grid_xgb.predict(X_test_scaled_OH)), grid_xgb.best_params_)

lightgbm mape: 45.92979626479157 {'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 150}


# CatBoost:

In [249]:
param_grid = {
    'n_estimators' : list(range(150, 400, 50)),
    # 'num_leaves': list(range(8, 16, 4)),
    # 'min_data_in_leaf': [10, 20, 40, 60, 100],
    'max_depth': [8, 12, 15],
    'learning_rate': [.3, 0.1, 0.01]}
    # 'bagging_freq': [3, 4, 5, 6, 7],
    # 'bagging_fraction': np.linspace(0.6, 0.95, 10),
    # 'reg_alpha': np.linspace(0.1, 0.95, 10),
    # 'reg_lambda': np.linspace(0.1, 0.95, 10)
                          
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better = False) 
model_cb = CatBoostRegressor()                     

    
grid_cb = GridSearchCV(model_cb, 
                        param_grid, 
                        scoring = scorer,
                        n_jobs=1,
                        pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
                        verbose=2)

grid_cb.fit(X_train_scaled_OH, y_train)

Fitting 3 folds for each of 45 candidates, totalling 135 fits
[CV] learning_rate=0.3, max_depth=8, n_estimators=150 ................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0:	learn: 1247.6164997	total: 97.4ms	remaining: 14.5s
1:	learn: 1095.6760521	total: 125ms	remaining: 9.23s
2:	learn: 998.6872896	total: 153ms	remaining: 7.48s
3:	learn: 954.3352896	total: 181ms	remaining: 6.59s
4:	learn: 918.6555790	total: 216ms	remaining: 6.26s
5:	learn: 895.8188568	total: 250ms	remaining: 5.99s
6:	learn: 860.9658836	total: 277ms	remaining: 5.67s
7:	learn: 839.0682885	total: 303ms	remaining: 5.37s
8:	learn: 829.7085094	total: 333ms	remaining: 5.22s
9:	learn: 813.7580781	total: 361ms	remaining: 5.05s
10:	learn: 802.5042431	total: 388ms	remaining: 4.9s
11:	learn: 791.6710770	total: 414ms	remaining: 4.76s
12:	learn: 782.6370998	total: 442ms	remaining: 4.66s
13:	learn: 760.6974925	total: 469ms	remaining: 4.55s
14:	learn: 750.8484772	total: 501ms	remaining: 4.51s
15:	learn: 746.8142747	total: 528ms	remaining: 4.43s
16:	learn: 739.4734215	total: 556ms	remaining: 4.35s
17:	learn: 734.2708912	total: 583ms	remaining: 4.27s
18:	learn: 724.7565862	total: 609ms	remaining: 4.2s
19

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.0s remaining:    0.0s


1:	learn: 1075.8051285	total: 60.2ms	remaining: 4.45s
2:	learn: 970.1124416	total: 90.6ms	remaining: 4.44s
3:	learn: 910.1093835	total: 119ms	remaining: 4.35s
4:	learn: 886.6206157	total: 139ms	remaining: 4.04s
5:	learn: 860.2703413	total: 176ms	remaining: 4.22s
6:	learn: 839.5730418	total: 205ms	remaining: 4.19s
7:	learn: 817.9507534	total: 233ms	remaining: 4.13s
8:	learn: 794.1957016	total: 261ms	remaining: 4.09s
9:	learn: 778.9545872	total: 293ms	remaining: 4.1s
10:	learn: 766.1444962	total: 321ms	remaining: 4.06s
11:	learn: 756.8603584	total: 350ms	remaining: 4.02s
12:	learn: 749.9920792	total: 377ms	remaining: 3.97s
13:	learn: 740.2085911	total: 405ms	remaining: 3.93s
14:	learn: 733.0529253	total: 435ms	remaining: 3.91s
15:	learn: 725.0172707	total: 472ms	remaining: 3.95s
16:	learn: 717.6365659	total: 516ms	remaining: 4.04s
17:	learn: 708.0797085	total: 559ms	remaining: 4.1s
18:	learn: 699.4092480	total: 596ms	remaining: 4.11s
19:	learn: 695.6403016	total: 622ms	remaining: 4.04s
2

...

291:	learn: 195.3588701	total: 27s	remaining: 5.37s
292:	learn: 194.7698280	total: 27.1s	remaining: 5.28s
293:	learn: 194.2406480	total: 27.2s	remaining: 5.18s
294:	learn: 193.8052575	total: 27.3s	remaining: 5.09s
295:	learn: 193.4471884	total: 27.4s	remaining: 5s
296:	learn: 192.9637659	total: 27.5s	remaining: 4.9s
297:	learn: 192.6430145	total: 27.6s	remaining: 4.81s
298:	learn: 192.2974608	total: 27.7s	remaining: 4.72s
299:	learn: 191.8301153	total: 27.7s	remaining: 4.62s
300:	learn: 191.5161830	total: 27.8s	remaining: 4.53s
301:	learn: 191.0161133	total: 27.9s	remaining: 4.44s
302:	learn: 190.7362091	total: 28s	remaining: 4.34s
303:	learn: 190.3391997	total: 28.1s	remaining: 4.25s
304:	learn: 189.9260024	total: 28.2s	remaining: 4.16s
305:	learn: 189.5480314	total: 28.3s	remaining: 4.07s
306:	learn: 189.0777305	total: 28.4s	remaining: 3.98s
307:	learn: 188.7377751	total: 28.5s	remaining: 3.88s
308:	learn: 188.2869153	total: 28.6s	remaining: 3.79s
309:	learn: 188.0168262	total: 28.7s

93:	learn: 391.8744946	total: 8.68s	remaining: 23.6s
94:	learn: 390.2327163	total: 8.76s	remaining: 23.5s
95:	learn: 388.5219382	total: 8.85s	remaining: 23.4s
96:	learn: 386.8211906	total: 8.94s	remaining: 23.3s
97:	learn: 385.1238179	total: 9.03s	remaining: 23.2s
98:	learn: 383.7360168	total: 9.12s	remaining: 23.1s
99:	learn: 382.0316490	total: 9.22s	remaining: 23.1s
100:	learn: 380.6564126	total: 9.31s	remaining: 22.9s
101:	learn: 378.6925033	total: 9.4s	remaining: 22.8s
102:	learn: 377.0572692	total: 9.49s	remaining: 22.8s
103:	learn: 374.8288300	total: 9.6s	remaining: 22.7s
104:	learn: 373.7027919	total: 9.69s	remaining: 22.6s
105:	learn: 372.4010053	total: 9.77s	remaining: 22.5s
106:	learn: 371.1574537	total: 9.87s	remaining: 22.4s
107:	learn: 368.9080516	total: 9.97s	remaining: 22.3s
108:	learn: 367.4926794	total: 10.1s	remaining: 22.2s
109:	learn: 365.5770902	total: 10.2s	remaining: 22.1s
110:	learn: 364.5528994	total: 10.3s	remaining: 22.1s
111:	learn: 362.6712400	total: 10.3s	

247:	learn: 236.6720482	total: 22.6s	remaining: 9.29s
248:	learn: 235.9821243	total: 22.7s	remaining: 9.2s
249:	learn: 235.2630526	total: 22.8s	remaining: 9.11s
250:	learn: 234.8073693	total: 22.9s	remaining: 9.02s
251:	learn: 234.1557490	total: 23s	remaining: 8.93s
252:	learn: 233.6262017	total: 23.1s	remaining: 8.84s
253:	learn: 233.0367054	total: 23.1s	remaining: 8.75s
254:	learn: 232.5648506	total: 23.2s	remaining: 8.65s
255:	learn: 231.8487672	total: 23.3s	remaining: 8.56s
256:	learn: 231.3214845	total: 23.4s	remaining: 8.47s
257:	learn: 230.4797920	total: 23.5s	remaining: 8.38s
258:	learn: 229.9832926	total: 23.6s	remaining: 8.29s
259:	learn: 229.3991374	total: 23.7s	remaining: 8.19s
260:	learn: 229.0990397	total: 23.8s	remaining: 8.1s
261:	learn: 228.5101573	total: 23.9s	remaining: 8.02s
262:	learn: 227.9188565	total: 24s	remaining: 7.92s
263:	learn: 227.2808971	total: 24s	remaining: 7.83s
264:	learn: 226.7157858	total: 24.1s	remaining: 7.74s
265:	learn: 225.9869518	total: 24.2s

49:	learn: 420.8547561	total: 26s	remaining: 52s
50:	learn: 418.3161018	total: 26.6s	remaining: 51.6s
51:	learn: 414.2216403	total: 27.1s	remaining: 51.1s
52:	learn: 409.2286738	total: 27.7s	remaining: 50.7s
53:	learn: 405.9890920	total: 28.3s	remaining: 50.2s
54:	learn: 402.7499571	total: 28.8s	remaining: 49.8s
55:	learn: 399.2477300	total: 29.4s	remaining: 49.4s
56:	learn: 395.8587842	total: 30s	remaining: 48.9s
57:	learn: 392.0723571	total: 30.6s	remaining: 48.5s
58:	learn: 388.0632522	total: 31.1s	remaining: 48s
59:	learn: 384.3655929	total: 31.7s	remaining: 47.5s
60:	learn: 381.8075195	total: 32.2s	remaining: 47s
61:	learn: 378.2694114	total: 32.8s	remaining: 46.5s
62:	learn: 376.3184233	total: 33.4s	remaining: 46.1s
63:	learn: 372.9531822	total: 34s	remaining: 45.7s
64:	learn: 370.7113270	total: 34.5s	remaining: 45.2s
65:	learn: 368.6686261	total: 35.1s	remaining: 44.7s
66:	learn: 366.7662177	total: 35.7s	remaining: 44.2s
67:	learn: 364.5061778	total: 36.3s	remaining: 43.8s
68:	l

KeyboardInterrupt: 

In [250]:
print("lightgbm mape:", mean_absolute_percentage_error(y_test, grid_cb.predict(X_test_scaled_OH)), grid_xgb.best_params_)

NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.